In [1]:
import tensorflow as tf
import math

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
x=mnist.train.images
y=mnist.train.labels

In [4]:
mnist.test.images.shape

(10000, 784)

In [7]:
print (x.shape)
print (y.shape)

(55000, 784)
(55000, 10)


In [8]:
X = tf.placeholder("float", [None,784])
Y_=tf.placeholder("float",[None,10])

In [9]:
# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 200
M = 100
N = 60
O = 30

In [10]:
# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([784, L], stddev=0.1))  # 784 = 28 * 28
B1 = tf.Variable(tf.zeros([L]))

W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.zeros([M]))

W3 = tf.Variable(tf.truncated_normal([M,N], stddev=0.1))
B3 = tf.Variable(tf.zeros([N]))

W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
B4 = tf.Variable(tf.zeros([O]))

W5 = tf.Variable(tf.truncated_normal([O, 10], stddev=0.1))
B5 = tf.Variable(tf.zeros([10]))

In [12]:
# variable learning rate
lr = tf.placeholder(tf.float32)
# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

In [13]:
# The model, with dropout at each layer

Y1 = tf.nn.relu(tf.matmul(X, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
Y2d = tf.nn.dropout(Y2, pkeep)

Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
Y3d = tf.nn.dropout(Y3, pkeep)

Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
Y4d = tf.nn.dropout(Y4, pkeep)

Ylogits = tf.matmul(Y4d, W5) + B5
Y = tf.nn.softmax(Ylogits)

In [15]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cost = tf.reduce_mean(cost)

In [16]:
# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [17]:
optimizer= tf.train.AdamOptimizer(lr).minimize(cost)

In [18]:
init =tf.global_variables_initializer()

In [19]:
# learning rate decay
max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations
#learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)

In [21]:
training_epochs = 25
batch_size = 100

In [23]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        c=0.0
        avg_cost=0.0
        # Loop over all batches
        for i in range(total_batch):
            
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
            
            batch_X, batch_Y = mnist.train.next_batch(batch_size)
            #print(batch_X.shape)
            _,c=sess.run([optimizer,cost], feed_dict={X:batch_X,Y_:batch_Y,lr: learning_rate,pkeep: 0.75})
            avg_cost += c / total_batch

        print("epchos {} cross entropy={}".format(epoch+1,avg_cost))
            
    print ("Optimization Finished!")

    # Test model
    #correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print( "Accuracy:", accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels,pkeep: 1.0}))

epchos 1 cross entropy=0.48591120634566626
epchos 2 cross entropy=0.2280315424705094
epchos 3 cross entropy=0.18030379728837453
epchos 4 cross entropy=0.15737649841064758
epchos 5 cross entropy=0.13691412672400485
epchos 6 cross entropy=0.1350135090371424
epchos 7 cross entropy=0.12155643020163884
epchos 8 cross entropy=0.11482642016288915
epchos 9 cross entropy=0.10793768431144683
epchos 10 cross entropy=0.10054949876767655
epchos 11 cross entropy=0.0973853578807955
epchos 12 cross entropy=0.09573796928013592
epchos 13 cross entropy=0.09024862816811287
epchos 14 cross entropy=0.08553889037109924
epchos 15 cross entropy=0.08209969356317416
epchos 16 cross entropy=0.08425726226022981
epchos 17 cross entropy=0.08686548179354177
epchos 18 cross entropy=0.07893212157068771
epchos 19 cross entropy=0.07572832425759946
epchos 20 cross entropy=0.0718018797420981
epchos 21 cross entropy=0.06866125213659623
epchos 22 cross entropy=0.07077309967738325
epchos 23 cross entropy=0.07580681973052293
e